In [1]:
param_id = '7782-000000-000001'
param_len = 54
param__ = 1708250866909
param_url = 'https://www.gates.com/gb/en/power-transmission/power-transmission-kits/micro-v-kits'

list_url = ['https://www.gates.com/bin/listing/products/variant?draw='+str(_+1)+'&columns%5B0%5D%5Bdata%5D=&columns%5B0%5D%5Bname%5D=partnumberauto_en_gb_string&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=&columns%5B1%5D%5Bname%5D=productnumber_en_gb_string&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=name_text_en_gb&columns%5B2%5D%5Bname%5D=name_text_en_gb&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=1&order%5B0%5D%5Bdir%5D=asc&start='+str(_*10)+'&length=10&search%5Bvalue%5D=&search%5Bregex%5D=false&id='+param_id+'&lang=en_gb&country=gb&fieldList=code_string%2Cpartnumberauto_en_gb_string%2Cpartnumberind_string%2Cproductnumber_en_gb_string%2Cname_text_en_gb&filters=&_='+str(param__+_+1) for _ in range(param_len)]

print('总数量：' + str(len(list_url)))
print()

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(len(list_url)):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()

        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(list_url[i],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                list_product_number = [_['code_string'].strip() for _ in resp['data']]
            
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'Page': i+1,
                                        'Url': [param_url+'.p.'+param_id+'.v.'+product_number+'.html' for product_number in list_product_number],
                                        'No.': [j+1 for j in range(len(list_product_number))]})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status != 'ok':
            df_temp = pd.DataFrame([{'Page': i+1}])
    
            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(str(i) +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Page', 'No.'], ascending=[True, True])
output_correct.to_excel('./menu-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Page'], ascending=[True])
    output_error.to_excel('./menu_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：54

19  <->  [ok] - 剩余数量：34 - 当前时间：17:51:16
4  <->  [ok] - 剩余数量：33 - 当前时间：17:51:16
11  <->  [ok] - 剩余数量：32 - 当前时间：17:51:16
18  <->  [ok] - 剩余数量：31 - 当前时间：17:51:16
15  <->  [ok] - 剩余数量：30 - 当前时间：17:51:16
3  <->  [ok] - 剩余数量：29 - 当前时间：17:51:16
5  <->  [ok] - 剩余数量：28 - 当前时间：17:51:16
12  <->  [ok] - 剩余数量：27 - 当前时间：17:51:16
8  <->  [ok] - 剩余数量：26 - 当前时间：17:51:17
9  <->  [ok] - 剩余数量：25 - 当前时间：17:51:17
6  <->  [ok] - 剩余数量：24 - 当前时间：17:51:17
2  <->  [ok] - 剩余数量：23 - 当前时间：17:51:17
23  <->  [ok] - 剩余数量：22 - 当前时间：17:51:17
20  <->  [ok] - 剩余数量：21 - 当前时间：17:51:17
24  <->  [ok] - 剩余数量：20 - 当前时间：17:51:17
7  <->  [ok] - 剩余数量：19 - 当前时间：17:51:17
25  <->  [ok] - 剩余数量：18 - 当前时间：17:51:17


KeyboardInterrupt
2024-02-19T09:51:18Z


KeyboardInterrupt: 